### **Install librairies offline**

In [1]:
import os
os.mkdir('./segmentation_models_pytorch')
!cp -r ../input/segmentation-models-pytorch/SegmentPModels/* ./segmentation_models_pytorch

In [2]:
%%capture
!pip install -q ./segmentation_models_pytorch/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4
!pip install -q ./segmentation_models_pytorch/efficientnet_pytorch-0.6.3/efficientnet_pytorch-0.6.3
!pip install -q ./segmentation_models_pytorch/timm-0.4.12-py3-none-any.whl
!pip install -q ./segmentation_models_pytorch/segmentation_models_pytorch-0.2.1-py3-none-any.whl

In [3]:
%%capture
!pip install --no-index --find-links="../input/highresnet/highresnet" highresnet


In [4]:
%%capture
!pip install --no-index --find-links="../input/torhio/TorchIO" torchio

### **Import librairies**

In [5]:
from pathlib import Path

import torchio as tio
import torch
import torchmetrics 
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from highresnet import HighRes3DNet

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gc
import nibabel as nib
import pickle
from tqdm.notebook import tqdm
import cv2
import os
import math

### **Preparation**

In [6]:
debug = False # set debug=False for test set

In [7]:
if debug : 
    ROOT_DIR = '../input/uw-madison-gi-tract-image-segmentation/train'
else: 
    ROOT_DIR ='../input/uw-madison-gi-tract-image-segmentation/test'
    #ROOT_DIR = '../input/test-eg-for-inference-mri-madison'
    


In [8]:
if debug: 
    df_original = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
else:
    df_original = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
    #df_original = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')

In [9]:
def prepare_df(df, root_dir):
    all_images = []
    for path in Path(root_dir).rglob('*.png'):

        parts = path.parts
        dict_path = {}
        case_str = parts[4][4:]
        day_str = parts[5].split('_')[1][3:]
        #dict_path['case'] = int(case_str)
        #dict_path['day'] = int(day_str)
        dict_path['slice'] = int(parts[7].split('_')[1])
        slice_str = '_'.join(parts[7].split('_')[0:2])
        #dict_path['PosixPath'] = path
        dict_path['path'] = str(path)
        dict_path['join_col'] = 'case'+ case_str + '_'+ 'day' + day_str + '_' + slice_str
        dict_path['height'] = int(dict_path['path'].split('/')[7].split('_')[2])
        dict_path['width'] = int(dict_path['path'].split('/')[7].split('_')[3])
        dict_path['subject'] = case_str + '_'+ day_str
        #dict_path['matching_str_inference'] = dict_path['subject'] + '_' +str(dict_path['slice'])
        all_images.append(dict_path)
    
    # Only contains 1/3 length of the orginal df(where each slice is repeated 3 times )
    df_1_3 = pd.DataFrame(all_images) 
    if df_1_3.empty:
        df = pd.DataFrame(columns={'id','class','predicted','slice','path','height','width','subject'})
        return df[['id','class','predicted','slice','path','height','width','subject']]
    # Get the final dataframe with the same length of the original one...
    else :
        df_final = pd.merge(df, df_1_3, left_on = 'id', right_on = 'join_col')
        df_final = df_final.drop('join_col', axis=1)
        df_final[['path','subject']] = df_final[['path','subject']].astype("Sparse[str]")
        df_final[['height','width','slice']]=df_final[['height','width','slice']].astype('int16')
        
        #I have to deactivate these 3 rows:here it is just to mimic the test set (for debugging)
        #df_final['segmentation'] =np.nan
        #df_final = df_final[['id','class','segmentation','slice','path','height','width','subject']]
        #df_final.columns = ['id','class','predicted','slice','path','height','width','subject']
        
        return df_final

In [10]:
df = prepare_df(df_original, ROOT_DIR)
#df.head(2)

In [11]:
def rearrange_df(df):
    """ 
    Rearrange data in the the prepared DataFrame (train_df or test_df).
    For each id (repeated 3 times),it creates 3 associated columns representing the segmentation 
    for the 3 classes:'large_bowel', 'small_bowel', 'stomach' 
    """
    if df.empty:
        df = pd.DataFrame(columns={'id','large_bowel','small_bowel','stomach','subject',
                                  'slice','path','width','height'})
        return df[['id','large_bowel','small_bowel','stomach','subject','slice','path','width','height']]
    else:
        # I keep only one id (so that it is not repeated 3 times)
        df_rearranged = pd.DataFrame({"id": df["id"][::3]}) 
        df_rearranged["large_bowel"] = df["predicted"][::3].values
        df_rearranged["small_bowel"] = df["predicted"][1::3].values
        df_rearranged["stomach"] = df["predicted"][2::3].values
        # Adjust the corresponding other columns
        #df_rearranged["case"] = df["case"][::3].values
        #df_rearranged["day"] = df["day"][::3].values
        df_rearranged["subject"] = df["subject"][::3].values
        df_rearranged["slice"] = df["slice"][::3].values
        df_rearranged["path"] = df["path"][::3].values
        df_rearranged["width"] = df["width"][::3].values
        df_rearranged["height"] = df["height"][::3].values
        #df_rearranged["matching_str_inference"] = df["matching_str_inference"][::3].values
        df_rearranged = df_rearranged.reset_index(drop=True)
        df_rearranged = df_rearranged.fillna("")
        return df_rearranged

In [12]:
df_rearranged = rearrange_df(df)
#df_rearranged.head(2)

In [13]:
# for debugging (from train set)
#df_rearranged = df_rearranged[0:7200] # 50 cases as the test set 

In [14]:
def create_list_subjects_slices(rearranged_df):
    if rearranged_df.empty:
        return [],[]
    else: 
        list_subjects_dict = []
        for subject, slices in list(rearranged_df.groupby('subject')['path']):
            dict_subject ={}
            list_slices_arrays = []
            for path_slice in slices.values:
                arr_img = cv2.imread(path_slice, cv2.IMREAD_ANYDEPTH)
                resized_arr_image = cv2.resize(arr_img, (128,128), interpolation=cv2.INTER_NEAREST)
                list_slices_arrays.append(resized_arr_image)
            dict_subject[subject] = list_slices_arrays
            list_subjects_dict.append(dict_subject)
    
        list_subjects_slices=[]
        list_subjects = []
        for x_dict in list_subjects_dict:
            for key_subject in x_dict:
                list_subjects_slices.append(x_dict[key_subject])
                list_subjects.append(key_subject)
        gc.collect()
        del list_subjects_dict 
        return list_subjects_slices, list_subjects

In [15]:
list_slices, list_subjects = create_list_subjects_slices(df_rearranged)

In [16]:
# Create niftii MRI images 
def create_3d_mri_images(list_all_slices, list_subjects, root):
    if (len(list_all_slices)==0)|(len(list_subjects)==0):
        os.makedirs('./mri_images/')
    else: 
        root_path = Path(root)
        root_path.mkdir(parents=True, exist_ok=True)
        list_mri_arrays=[]
        for subject, slices_subject in zip(list_subjects,list_all_slices):
            mri_subject = np.asarray(slices_subject)
            mri_subject = np.swapaxes(mri_subject, 0, 2) # (num_slices,w,h) --> (h, w, num_slices)
            mri_subject = np.swapaxes(mri_subject, 0, 1) # (h,w, num_slices) --> (w,h,num_slices)
            mri_subject_nifti = nib.Nifti1Image(mri_subject, affine=np.eye(4))
            nib.save(mri_subject_nifti, root+subject+'.nii.gz')
        gc.collect()

In [17]:
create_3d_mri_images(list_slices,list_subjects, root= './mri_images/')

del list_slices[:]
del list_slices
del list_subjects[:]
del list_subjects
gc.collect()
gc.collect(generation=2)

0

### **Dataset & Model** 

In [18]:
path_test = Path('./mri_images')

if not os.listdir(path_test):
    pass
else: 
    subjects_test_paths = sorted(list(path_test.glob("*nii.gz")))
    subjects_test = []

    for subject_path in subjects_test_paths:
        subject = tio.Subject({"MRI":tio.ScalarImage(subject_path)})
        subjects_test.append(subject)

In [19]:
# I will need it because I'll use CropOrPad to (128,128,144)
# But I need the real number of slices, mainly for the padded slices (during running inference code...)
if not os.listdir(path_test):
    pass
else: 
    list_size = []
    for element in subjects_test:
        list_size.append(element['MRI']['data'].shape[3])
    list_size_scans = np.array(list_size).astype(np.int16)
    del list_size
    list_size_scans = list(list_size_scans)
    #np.unique(list_size_scans,return_counts=True)

In [20]:
# Histogram Standardization
if not os.listdir(path_test):
    pass
else: 
    histogram_landmarks_path = './landmarks.npy' # juste path 

    landmarks = tio.HistogramStandardization.train(
        subjects_test_paths,
        output_path=histogram_landmarks_path,)
    np.set_printoptions(suppress=True, precision=3)
    print('\nTrained landmarks:', landmarks)

In [21]:
if not os.listdir(path_test):
    pass
else: 
    process = tio.Compose([
        tio.CropOrPad((128,128, 144)),
        tio.HistogramStandardization({'MRI': landmarks}),
        tio.ZNormalization(masking_method=tio.ZNormalization.mean)])
    test_transform = process

In [22]:
# Create the TorchIo Test Dataset
if not os.listdir(path_test):
    pass
else: 
    test_dataset = tio.SubjectsDataset(subjects_test, transform=test_transform)

In [23]:
class SegmentMadisonOrgans(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        self.model = HighRes3DNet(in_channels=1, out_channels=4)
        self.loss_fn = DiceLoss(mode ='multiclass',classes=4, from_logits=True)
         
        self.metric = torchmetrics.JaccardIndex(num_classes=4)
        self.lr = 5e-4
    
    def forward(self, data):
        pred = self.model(data)
        return pred
    
    def training_step(self, batch, batch_idx):
        img = batch["MRI"]["data"] # in TorhIo, we need to acess the label entry "MRI", then "data"
        mask = batch["Label"]["data"][:,0]  # Remove single channel as Loss expects NxHxW
        mask = mask.long()
        
        pred = self(img)
        loss = self.loss_fn(pred, mask)
        iou = self.metric(pred,mask)
        self.log("Train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('Train_iou',iou, on_step=False, on_epoch=True, prog_bar=True)
        
        return loss
    
        
    def validation_step(self, batch, batch_idx):
        img = batch["MRI"]["data"]
        mask = batch["Label"]["data"][:,0]  # Remove single channel --> NxHxW
        mask = mask.long()
        
        pred = self(img)
        loss = self.loss_fn(pred, mask)
        iou = self.metric(pred,mask)
        self.log("Val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('Val_iou',iou, on_step=False, on_epoch=True, prog_bar=True)

        return loss
    
    def configure_optimizers(self):
        opt=torch.optim.AdamW(self.parameters(),lr=self.lr)
        scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=3,verbose=True)
        return {'optimizer':opt,'scheduler':scheduler}

In [24]:
if not os.listdir(path_test):
    pass
else: 
    model = SegmentMadisonOrgans.load_from_checkpoint('../input/weights-128-3d/epoch40-step6970.ckpt')
    model = model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device);

### **Predictions** 

In [25]:
# Predictions without TTA: Test Time Augmentation
if not os.listdir(path_test):
    pass
else: 
    all_preds =[]
    for idx in tqdm(range(0,len(test_dataset))):
        imgs = test_dataset[idx]["MRI"]["data"]
        grid_sampler = tio.inference.GridSampler(test_dataset[idx], 96, (8, 8, 8))
        aggregator = tio.inference.GridAggregator(grid_sampler)
        patch_loader = torch.utils.data.DataLoader(grid_sampler, batch_size=4)
        with torch.no_grad():
            for patches_batch in patch_loader:
                input_tensor = patches_batch['MRI']["data"].to(device)  # Get batch of patches
                locations = patches_batch[tio.LOCATION]  # Get locations of patches
                pred = model(input_tensor)  # Compute prediction
                aggregator.add_batch(pred, locations)  # Combine predictions to volume
            output_tensor = aggregator.get_output_tensor()
            del aggregator
            gc.collect()
            pred = output_tensor.argmax(0)
            del output_tensor
            gc.collect()
            if (list_size_scans[idx] != 144):
                size = list_size_scans[idx]
                pred = pred[:,:,int((144-size)/2): -int((144-size)/2)]
            all_preds.append(pred.to(torch.int8))
            del pred
            gc.collect()
            torch.cuda.empty_cache()

## **TTA**

In [26]:
if not os.listdir(path_test):
    pass
else: 
    num_augmentations = 4

    #flip_lateral = tio.RandomFlip(flip_probability=1)
    #flip_Right = tio.RandomFlip(axes=['Right'], flip_probability=0.8)

    augment = tio.RandomFlip(flip_probability=1)

    all_preds_aug =[]
    for idx in tqdm(range(0,len(test_dataset))):
        results = []
        for _ in range(num_augmentations):
            augmented = augment(test_dataset[idx])
            grid_sampler = tio.inference.GridSampler(augmented, 96, (8, 8, 8))
            aggregator = tio.inference.GridAggregator(grid_sampler)
            patch_loader = torch.utils.data.DataLoader(grid_sampler, batch_size=4)
    
        with torch.no_grad():
            for patches_batch in patch_loader:
                input_tensor = patches_batch['MRI']["data"].to(device)  # Get batch of patches
                locations = patches_batch[tio.LOCATION]  # Get locations of patches
                pred = model(input_tensor)  # Compute prediction
                aggregator.add_batch(pred, locations)  # Combine predictions to volume
            output_tensor = aggregator.get_output_tensor()
            del aggregator
            gc.collect()
            pred = output_tensor.argmax(0)
            del output_tensor
            gc.collect()
            # I add the label to tioSubject(using the MRI image data)
            # NB: augmented.MRI.affine access the original shape of MRI, not the padded or corpped
            # So: no need to loop over scans shape... like it was with simple predictions above
            lm_temp = tio.LabelMap(tensor=torch.rand(1,1,1,1), affine=augmented.MRI.affine)
            augmented.add_image(lm_temp, 'Label')
            augmented.Label.set_data(torch.unsqueeze(pred,dim=0))# TorchIo Subject has BS dim
            back = augmented.apply_inverse_transform(warn=True)
            results.append(back.Label.data[0]) # to get rid of BS dimension
        results.append(all_preds[idx])
        result = torch.stack(results).long()
        tta_result_tensor = result.mode(dim=0).values
        
        # Here we don't need this because MRI.affine access the original size (see above)
        #if (list_size_scans[idx] != 144):
            #size = list_size_scans[idx]
            #tta_result_tensor= tta_result_tensor[:,:,int((144-size)/2): -int((144-size)/2)]
        all_preds_aug.append(tta_result_tensor.to(torch.int8))
        del tta_result_tensor
        gc.collect()
        torch.cuda.empty_cache()

In [27]:
# Becasue of TTA
if not os.listdir(path_test):
    pass
else: 
    del all_preds
    gc.collect()

### **Rearrange Predictions for submission**

In [28]:
# list paths of transformed niftii test images (/output)
# we got it (the sorted list before creating the TorchIo Dataset)
# subjects_test_paths
if not os.listdir(path_test):
    pass
else: 
    list_cases = [element.parts[1][:-7] for element in subjects_test_paths]

In [29]:
# Create a dictionary that associates each case to its 3D mask (128,128,144) or (128,128,80)
# Still with the resized masks
if not os.listdir(path_test):
    pass
else: 
    dict_cases_preds = {}

    #cases and all_preds or (all_preds_aug if TTA) are in the same order since test_dataset was created from the sorted list..
    for case,pred in zip(list_cases, all_preds_aug):
        dict_cases_preds[case] = [pred[:,:,i].numpy() for i in range(0,pred.shape[2])]

In [30]:
df_reconst= pd.DataFrame(columns={'case', 'pred','slice'})

In [31]:
# Compute the case column from test cases (first from 3D scans not from .csv)
# I'll restore 2D slices form 3D masks
if not os.listdir(path_test):
    pass
else: 
    column_case = []
    for case in list(dict_cases_preds.keys()):
        column_case += [case]*len(dict_cases_preds[case])

In [32]:
# Compute the predictions column (extract 2D predictions form 3D predictions)
if not os.listdir(path_test):
    pass
else: 
    preds_col = []
    for preds_case in list(dict_cases_preds.values()):
        preds_col += preds_case
#print(len(preds_col))

In [33]:
if not os.listdir(path_test):
    pass
else: 
    df_reconst['case'] = column_case
    df_reconst['pred'] = preds_col

In [34]:
# Compute the large_bowel, small_bowel & stomach predictions
if not os.listdir(path_test):
    pass
else: 
    df_reconst['large_bowel'] = df_reconst.apply(lambda x: (x['pred']==1).astype('int8'), axis=1)
    df_reconst['small_bowel'] = df_reconst.apply(lambda x: (x['pred']==2).astype('int8'), axis=1)
    df_reconst['stomach'] = df_reconst.apply(lambda x: (x['pred']==3).astype('int8'), axis=1)

In [35]:
# restore the form of the submission file. For each slice I have 3 rows (one for large_bowel
# one row for small_bowle and one row for stomach)
if not os.listdir(path_test):
    pass
else: 
    list_case = list(df_reconst['case'])
    list_subject_mult3 =[]
    for element in list_case:
        list_subject_mult3 += [element]*3

In [36]:
if not os.listdir(path_test):
    pass
else: 
    df_final = pd.DataFrame(columns={'class','segmentation','subject','slice','width', 'height'})
    df_final['subject'] = list_subject_mult3
#len(df_final)

In [37]:
# Segmentation column
if not os.listdir(path_test):
    pass
else: 
    df_final['segmentation'][::3] = df_reconst['large_bowel']
    df_final['segmentation'][1::3] = df_reconst['small_bowel']
    df_final['segmentation'][2::3] = df_reconst['stomach']

In [38]:
# class column as submission file
if not os.listdir(path_test):
    pass
else: 
    df_final['class'][::3] = ['large_bowel']*len(df_reconst)
    df_final['class'][1::3] = ['small_bowel']*len(df_reconst)
    df_final['class'][2::3] = ['stomach']*len(df_reconst)

In [39]:
# Extract slice number from submission file (because we have no idea if the slices are in order
# for example when we have 80 slices: they may be, 1,2,4,8,10 and not 1,2,3,4,5...)
if not os.listdir(path_test):
    pass
else: 
    list_subject_slices= list(df_rearranged.groupby('subject')['slice']) # liste des tuples ('suject','series of slices')

    list_dict_subject_slices=[]
    for subject, series_slices in list_subject_slices:
        dict_subject_slices ={}
        dict_subject_slices[subject] = list(series_slices.values)
        list_dict_subject_slices.append(dict_subject_slices)

In [40]:
# Activate this for inference competition test set

# slice column
if not os.listdir(path_test):
    pass
else: 
    for element in list_dict_subject_slices:
        list_slices_mult3=[]
        for el in list(element.values())[0]: # loop over slices of a subject
            list_slices_mult3 += [el]*3 # because 3rows for each slice: large_bowel, small and stomach
            # list(element.keys())[0] is an elment for example '101_20'
            # and list(element.values())[0] is for example [1,2,3,.....,144]
        if len(df_final['slice'].loc[df_final.subject==list(element.keys())[0]])!= len(list_slices_mult3):
            print(element.keys())
        df_final['slice'].loc[df_final.subject==list(element.keys())[0]]= list_slices_mult3
    

In [41]:
# id column
if not os.listdir(path_test):
    pass
else: 
    df_final['id']= df_final.apply(lambda x: 'case'+x['subject'].split('_')[0]+'_day'+x['subject'].split('_')[1]+'_slice_'+str(x['slice']).zfill(4),axis=1)

In [42]:
if not os.listdir(path_test):
    pass
else: 
    # width column (for inference)
    list_subject_width= list(df_rearranged.groupby('subject')['width']) 
    list_dict_subject_widths=[]
    for subject, series_widths in list_subject_width:
        dict_subject_width ={}
        dict_subject_width[subject] = list(series_widths.values)
        list_dict_subject_widths.append(dict_subject_width)


    for element in list_dict_subject_widths:
        list_width_mult3=[]
        for el in list(element.values())[0]:
            list_width_mult3 += [el]*3
        df_final['width'].loc[df_final.subject==list(element.keys())[0]]= list_width_mult3
    del list_dict_subject_widths

In [43]:
# height column for inference
if not os.listdir(path_test):
    pass
else: 
    list_subject_height= list(df_rearranged.groupby('subject')['height']) # list of tuples ('suject','series of slices')
    list_dict_subject_heights=[]
    for subject, series_heights in list_subject_height:
        dict_subject_height ={}
        dict_subject_height[subject] = list(series_heights .values)
        list_dict_subject_heights.append(dict_subject_height)

    for element in list_dict_subject_heights:
        list_height_mult3=[]
        for el in list(element.values())[0]:
            list_height_mult3 += [el]*3
        df_final['height'].loc[df_final.subject==list(element.keys())[0]]= list_height_mult3
    del list_dict_subject_heights

In [44]:
#df_final.head(2)

In [45]:
# add resized segmentation (to the original size)
if not os.listdir(path_test):
    pass
else: 
    df_final['segment_resized'] = df_final.apply(lambda x: cv2.resize(x['segmentation'], (x['width'], x['height']),interpolation=cv2.INTER_NEAREST), axis=1)

In [46]:
# rle Encoding

def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.flatten() # était img.T.flatten() (car height and width reversed)
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [47]:
if not os.listdir(path_test):
    pass
else: 
    df_final['predicted'] = df_final.apply(lambda x:mask2rle(x['segment_resized']), axis=1 )

In [48]:
if not os.listdir(path_test):
    pass
else: 
    df_final.drop(['height','width','subject','segmentation','segment_resized','slice'], axis=1, inplace= True)

In [49]:
if not os.listdir(path_test):
    sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
    sub_df.columns = ['id','class','predicted']
else: 
    sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
    sub_df.drop(['predicted'], axis=1, inplace=True)
    sub_df = sub_df.merge(df_final, on=['id','class'])
    sub_df =sub_df[['id','class','predicted']]

In [50]:
sub_df.to_csv('submission.csv',index=False)
display(sub_df.head(5))

,id,class,predicted
0,case123_day20_slice_0001,large_bowel,NaN
1,case123_day20_slice_0001,small_bowel,NaN
2,case123_day20_slice_0001,stomach,NaN
3,case123_day20_slice_0002,large_bowel,NaN
4,case123_day20_slice_0002,small_bowel,NaN
